# Simple CNN Model for classification

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('/kaggle/input/drone-and-not-drone/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[224, 224, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

cnn.fit(training_set, epochs = 25)

Found 127 images belonging to 2 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/25


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/4 ━━━━━━━━━━━━━━━━━━━━ 1:54 38s/step - accuracy: 0.4688 - loss: 0.7345

I0000 00:00:1714736390.995877     115 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1714736391.012791     115 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4/4 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.4534 - loss: 8.5504 
Epoch 2/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 29s 40ms/step - accuracy: 0.5807 - loss: 0.6728
Epoch 3/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 30s 41ms/step - accuracy: 0.5902 - loss: 0.6493
Epoch 4/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 30s 41ms/step - accuracy: 0.5928 - loss: 0.6477
Epoch 5/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - accuracy: 0.6450 - loss: 0.6095
Epoch 6/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 28s 40ms/step - accuracy: 0.7804 - loss: 0.5557
Epoch 7/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 29s 41ms/step - accuracy: 0.6983 - loss: 0.5590
Epoch 8/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 30s 41ms/step - accuracy: 0.7298 - loss: 0.5149
Epoch 9/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 30s 40ms/step - accuracy: 0.7175 - loss: 0.5080
Epoch 11/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 29s 41ms/step - accuracy: 0.8365 - loss: 0.4101
Epoch 12/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 30s 40ms/step - accuracy: 0.7857 - loss: 0.4018
Epoch 13/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 32s 44ms/step - accuracy: 0.7854 - loss: 0.4062
E

NameError: name 'model' is not defined

In [4]:
import os, numpy as np
from tensorflow.keras.preprocessing import image

test_folder = "/kaggle/input/drone-and-not-drone/val/Drone/"

print(training_set.class_indices)

test_generator = train_datagen.flow_from_directory(
    '/kaggle/input/drone-and-not-drone/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

loss, accuracy = cnn.evaluate(test_generator)
print(f'Test accuracy: {accuracy}')

{'Drone': 0, 'Not Drone': 1}
Found 11 images belonging to 1 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.7273 - loss: 1.1845
Test accuracy: 0.7272727489471436


# Fine tuning ResNet50 Model for classification

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

resnet_base = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in resnet_base.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(resnet_base.output)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=resnet_base.input, outputs=predictions)

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/drone-and-not-drone/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

history = model.fit(train_generator, epochs=25)

test_generator = train_datagen.flow_from_directory(
    '/kaggle/input/drone-and-not-drone/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy}')

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Found 127 images belonging to 2 classes.
Epoch 1/25
1/4 ━━━━━━━━━━━━━━━━━━━━ 2:24 48s/step - accuracy: 0.4688 - loss: 0.7015

W0000 00:00:1714738571.348101     113 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3/4 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.4922 - loss: 0.7399 

W0000 00:00:1714738580.261883     116 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4/4 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step - accuracy: 0.4906 - loss: 0.7408
Epoch 2/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 28s 94ms/step - accuracy: 0.5184 - loss: 0.7010
Epoch 3/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 30s 96ms/step - accuracy: 0.5676 - loss: 0.6917
Epoch 4/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 28s 87ms/step - accuracy: 0.6376 - loss: 0.6724
Epoch 5/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 28s 87ms/step - accuracy: 0.6482 - loss: 0.6670
Epoch 6/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 32s 92ms/step - accuracy: 0.6083 - loss: 0.6776
Epoch 7/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 32s 87ms/step - accuracy: 0.5487 - loss: 0.6702
Epoch 8/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 32s 88ms/step - accuracy: 0.4882 - loss: 0.6760
Epoch 9/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 86ms/step - accuracy: 0.5684 - loss: 0.6598
Epoch 10/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 32s 97ms/step - accuracy: 0.5236 - loss: 0.6716
Epoch 11/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 28s 95ms/step - accuracy: 0.7192 - loss: 0.6459
Epoch 12/25
4/4 ━━━━━━━━━━━━━━━━━━━━ 28s 99ms/step - accuracy: 0.5944 - loss: 0.6367
Ep

W0000 00:00:1714739321.892119     116 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
